In [ ]:
%run strain_report_dev.py

In [ ]:
import seaborn as sns
sns.heatmap(strain.signals['mCherry/butter'].dropna().head(25))

In [ ]:
foo = strain.signals['mCherry/butter'].dropna()

In [ ]:
fig, ax = plt.subplots(figsize=(10,3))
plt.plot(foo.iloc[7])

In [ ]:
foo.iloc[7]

In [ ]:
import seaborn as sns
sns.heatmap(strain.flavin_processed, cmap='RdBu', robust=True)

In [ ]:
strain.flavin_processed

In [ ]:
strain.flavin_processed.iloc[72]

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(strain.flavin_processed.iloc[0:100], cmap='RdBu', robust=True)

In [ ]:
sns.heatmap(strain.flavin_acfs.iloc[0:100], cmap='RdBu', robust=True)

In [ ]:
sns.heatmap(wildtype.flavin_acfs.iloc[0:100], cmap='RdBu', robust=True)

In [ ]:
sns.clustermap(
    strain.flavin_processed,
    col_cluster=False,
    cmap='RdBu_r',
    robust=True,
)

In [ ]:
sns.clustermap(
    wildtype.flavin_processed,
    col_cluster=False,
    cmap='RdBu_r',
    robust=True,
)

In [ ]:
import pandas as pd
row_colors = 'r' * len(wildtype.flavin_processed) + 'b' * len(strain.flavin_processed)
#bodge
s = strain.flavin_processed
s.columns = wildtype.flavin_processed.columns
sns.clustermap(
    pd.concat([wildtype.flavin_processed, s], axis=0, ignore_index=True),
    col_cluster=False,
    cmap='RdBu_r',
    robust=True,
    row_colors=row_colors,
)

In [ ]:
strain.flavin_processed

In [ ]:
#sns.heatmap(signal_births_processed.iloc[0:50])
sns.heatmap(strain.births_processed)

In [ ]:
celliloc = 27
t = wildtype.flavin_processed.columns
y = wildtype.flavin_processed.iloc[celliloc].to_numpy()
b = wildtype.births_processed.iloc[celliloc].to_numpy()
p = wildtype.peaks.iloc[celliloc].to_numpy()
p_mask = np.ma.make_mask(p)

def acorr(y):
    norm = y - np.mean(y)
    var = np.var(y)
    acorr = np.correlate(norm, norm, 'full')[len(norm)-1:]
    acorr = acorr / var / len(norm)
    return acorr

plt.subplots(figsize=(20,5))
#plt.plot(y)
single_birth_plot(t, y, birth_mask=b)
plt.plot(t[p_mask], y[p_mask], 'x', color='r')

plt.subplots()
from scipy.signal import find_peaks
a = acorr(y)
acorr_peaks, _ = find_peaks(a)
plt.plot(a)
plt.plot(acorr_peaks, a[acorr_peaks], 'x')

print(acorr_peaks)
print(acorr_peaks[0] / 12)

In [ ]:
5/60 * mask_to_intervals(pd.DataFrame(b).T)

In [ ]:
acorr_peaks / 12

In [ ]:
sns.heatmap(signal_births_processed)

In [ ]:
sns.heatmap(signal_peaks)

In [ ]:
# Proxies for synchrony
med = standardscaler.as_function(signal_flavin_processed).median()

plt.subplots()
plt.plot(med)
plt.title('median time series')

plt.subplots()
plt.plot(acorr(med))
plt.title('autocorrelation')
plt.xlabel('lag')
plt.ylabel('corr')

fft_freqs, fft_power = fft.as_function(pd.DataFrame(med).T)
plt.subplots()
plt.plot(fft_freqs.to_numpy()[0], fft_power.to_numpy()[0])
plt.title('fourier spectrum')
plt.xlabel('freq')
plt.ylabel('power')

In [ ]:
# Auto-correlation

In [ ]:
# Find peaks in norm_corr, plot on heatmap
from postprocessor.core.processes.findpeaks import findpeaksParameters, findpeaks
p = findpeaks.as_function(norm_corr, prominence = 0.09)

plt.subplots(figsize=(15,15))
heatmap(p, 'peak', xtick_step=12, xlabel='tp lag')

In [ ]:
# Plot where acf peaks are -- helps with calibrating findpeaks parameters
pos = 16
plt.plot(norm_corr.iloc[pos])
p_mask = np.ma.make_mask(p.to_numpy())
plt.plot(norm_corr.columns[p_mask[pos]], norm_corr.iloc[pos][p_mask[pos]], 'x')

In [ ]:
# Distribution of cycle lengths, estimated from acf

# get interval lengths then get just the first one
# (adapted from first line in mask_to_intervals)
temp_df = p.apply(lambda x: np.diff(np.where(np.array(x) > 0))[0][0], axis=1)
# convert from time points to minutes
temp_df = temp_df.apply(lambda x: x * 5)
# histogram
binsize = 5
plt.hist(temp_df, bins=np.arange(min(temp_df), max(temp_df) + binsize, binsize))

XY coords of traps

In [ ]:
strain.signals['flavin/butter']

In [ ]:
cell_ids = strain.signals['flavin/butter'].index.to_list()
x_coords = []
y_coords = []
for position, trap, cell_label in cell_ids:
    traps = grouper.tilelocs[position]
    x_coords.append(traps[trap,0])
    y_coords.append(traps[trap,1])

In [ ]:
xy_df = pd.DataFrame([x_coords, y_coords])
xy_df = xy_df.T
xy_df.columns = ['x', 'y']
xy_df.index = strain.signals['flavin/butter'].index

In [ ]:
xy_df

In [ ]:
xy_df.to_csv('20016_by4741_XYcoordinates.csv', index=True)